# MuRIL

In [1]:
import random
random.seed(10)

import numpy as np
X_Embed_train = np.load('Embeddings/Kan_MuRIL_train.npy')
X_Embed_val = np.load('Embeddings/Kan_MuRIL_val.npy')

In [2]:
Y_raw_train = np.load("Embeddings/Kan/Split2750and250/Kan_Train_label_2750.npy")
Y_raw_val = np.load("Embeddings/Kan/Split2750and250/Kan_val_label_250.npy")

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_raw_train)
Y_val = le.fit_transform(Y_raw_val)
Y_train.shape, Y_val.shape

((2750,), (250,))

In [4]:
def WR_Wavelet(signal, sigma):
    (cA, cD) = pywt.dwt(signal,'haar')
    t=2*sigma  
    wt=(np.abs(cD))>t                     #only the coefficient values greater than the threshold values are taken
    #print(cA.shape , wt.shape ,'\n')
    p = np.multiply(cD,wt)
    rw=pywt.idwt(cA, p,'haar')
    return rw

In [5]:
import pywt
sigma = 0.1

MuRIL_train_list = []
for i in range(len(X_Embed_train)):
    temp = WR_Wavelet(X_Embed_train[i], sigma)
    MuRIL_train_list.append(temp)
    
MuRIL_val_list = []
for i in range(len(X_Embed_val)):
    temp = WR_Wavelet(X_Embed_val[i],sigma)
    MuRIL_val_list.append(temp)

In [6]:
X_MuRIL_Wavlet_Denoise_Embed_train = np.asarray(MuRIL_train_list)
X_MuRIL_Wavlet_Denoise_Embed_val = np.asarray(MuRIL_val_list)

In [7]:
from Models.MLAlgos import SVectorMachine, Dtree, Rforest, NB, KNN, DeepForest

SVM_MuRIL_WL_Denoise_Result = SVectorMachine(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, Cval = 1000, deguree = 1)
Dtree_MuRIL_WL_Denoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=5, split=2)
Rforest_MuRIL_WL_Denoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=10, split= 26)
NB_MuRIL_WL_Denoise_Result = NB(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val)
KNN_MuRIL_WL_Denoise_Result = KNN(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val)
Deep_MuRIL_WL_Denoise_Result = DeepForest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val)

[2021-11-26 13:54:54.582] Start to fit the model:
[2021-11-26 13:54:54.583] Fitting cascade layer = 0 
[2021-11-26 13:55:03.644] layer = 0  | Val Acc = 46.618 % | Elapsed = 9.061 s
[2021-11-26 13:55:03.659] Fitting cascade layer = 1 
[2021-11-26 13:55:11.343] layer = 1  | Val Acc = 47.673 % | Elapsed = 7.683 s
[2021-11-26 13:55:11.354] Fitting cascade layer = 2 
[2021-11-26 13:55:18.739] layer = 2  | Val Acc = 47.745 % | Elapsed = 7.385 s
[2021-11-26 13:55:18.751] Fitting cascade layer = 3 
[2021-11-26 13:55:26.718] layer = 3  | Val Acc = 47.964 % | Elapsed = 7.967 s
[2021-11-26 13:55:26.729] Fitting cascade layer = 4 
[2021-11-26 13:55:34.050] layer = 4  | Val Acc = 46.982 % | Elapsed = 7.321 s
[2021-11-26 13:55:34.050] Early stopping counter: 1 out of 2
[2021-11-26 13:55:34.063] Fitting cascade layer = 5 
[2021-11-26 13:55:41.478] layer = 5  | Val Acc = 46.982 % | Elapsed = 7.415 s
[2021-11-26 13:55:41.478] Early stopping counter: 2 out of 2
[2021-11-26 13:55:41.478] Handling early s

In [8]:
import pandas as pd
MuRIL_WaveFilter_AFTR_Denoising= pd.DataFrame({'SVM' : SVM_MuRIL_WL_Denoise_Result, 
            'DTree' : Dtree_MuRIL_WL_Denoise_Result,
            'Rforest' : Rforest_MuRIL_WL_Denoise_Result,
            'NB' : NB_MuRIL_WL_Denoise_Result,
            'KNN' : KNN_MuRIL_WL_Denoise_Result,
            'DeepForest': Deep_MuRIL_WL_Denoise_Result})
MuRIL_WaveFilter_AFTR_Denoising.index = ['accuracy', 'precision', 'f1', 'recall']
MuRIL_WaveFilter_AFTR_Denoising = MuRIL_WaveFilter_AFTR_Denoising.transpose()
MuRIL_WaveFilter_AFTR_Denoising

,accuracy,precision,f1,recall
SVM,46.4000%,45.2095%,45.6054%,46.4000%
DTree,42.4000%,40.7652%,40.3683%,42.4000%
Rforest,51.2000%,51.3668%,50.1676%,51.2000%
NB,37.6000%,33.0762%,34.0614%,37.6000%
KNN,45.6000%,46.2718%,44.4498%,45.6000%
DeepForest,50.0000%,49.0178%,47.9370%,50.0000%


# LABSE

In [1]:
import random 
random.seed(10)

In [2]:
import numpy as np
X_Embed_train = np.load('Embeddings/Kan_MuRIL_train.npy')
X_Embed_val = np.load('Embeddings/Kan_MuRIL_val.npy')

In [3]:
Y_raw_train = np.load("Embeddings/Kan/Split2750and250/Kan_Train_label_2750.npy")
Y_raw_val = np.load("Embeddings/Kan/Split2750and250/Kan_val_label_250.npy")

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_raw_train)
Y_val = le.fit_transform(Y_raw_val)
Y_train.shape, Y_val.shape

((2750,), (250,))

In [5]:
import pywt
def WR_Wavelet(signal, sigma):
    (cA, cD) = pywt.dwt(signal,'haar')
    t=4*sigma  
    wt=(np.abs(cD))>t                     #only the coefficient values greater than the threshold values are taken
    #print(cA.shape , wt.shape ,'\n')
    p = np.multiply(cD,wt)
    rw=pywt.idwt(cA, p,'haar')
    return rw

In [6]:
sigma = 0.2

train_list = []
for i in range(len(X_Embed_train)):
    temp = WR_Wavelet(X_Embed_train[i], sigma)
    train_list.append(temp)
    
val_list = []
for i in range(len(X_Embed_val)):
    temp = WR_Wavelet(X_Embed_val[i],sigma)
    val_list.append(temp)

In [7]:
X_Wavlet_Denoise_Embed_train = np.asarray(train_list)
X_Wavlet_Denoise_Embed_val = np.asarray(val_list)

from Models.MLAlgos import SVectorMachine, Dtree, Rforest, NB, KNN, DeepForest
SVM_WL_Denoise_Result = SVectorMachine(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,Cval=1000, deguree=1, kernelu ='rbf')
Dtree_WL_Denoise_Result = Dtree(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,deepth =5 ,split=2)
Rforest_WL_Denoise_Result = Rforest(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,deepth =10 ,split=26)
NB_WL_Denoise_Result = NB(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val)
KNN_WL_Denoise_Result = KNN(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val)
Deep_WL_Denoise_Result = DeepForest(X_Embed_train, Y_train, X_Embed_val, Y_val)

[2021-11-26 18:51:53.562] Start to fit the model:
[2021-11-26 18:51:53.562] Fitting cascade layer = 0 
[2021-11-26 18:52:00.385] layer = 0  | Val Acc = 46.582 % | Elapsed = 6.823 s
[2021-11-26 18:52:00.398] Fitting cascade layer = 1 
[2021-11-26 18:52:07.226] layer = 1  | Val Acc = 47.818 % | Elapsed = 6.828 s
[2021-11-26 18:52:07.237] Fitting cascade layer = 2 
[2021-11-26 18:52:13.995] layer = 2  | Val Acc = 48.255 % | Elapsed = 6.758 s
[2021-11-26 18:52:14.005] Fitting cascade layer = 3 
[2021-11-26 18:52:20.774] layer = 3  | Val Acc = 47.491 % | Elapsed = 6.769 s
[2021-11-26 18:52:20.774] Early stopping counter: 1 out of 2
[2021-11-26 18:52:20.784] Fitting cascade layer = 4 
[2021-11-26 18:52:27.890] layer = 4  | Val Acc = 48.073 % | Elapsed = 7.106 s
[2021-11-26 18:52:27.890] Early stopping counter: 2 out of 2
[2021-11-26 18:52:27.890] Handling early stopping
[2021-11-26 18:52:27.892] The optimal number of layers: 3
[2021-11-26 18:52:27.895] Start to evalute the model:
[2021-11-26

In [8]:
import pandas as pd
WaveFilter_AFTR_Denoising= pd.DataFrame({'SVM' : SVM_WL_Denoise_Result, 
            'DTree' : Dtree_WL_Denoise_Result,
            'Rforest' : Rforest_WL_Denoise_Result,
            'NB' : NB_WL_Denoise_Result,
            'KNN' : KNN_WL_Denoise_Result,
            'DeepForest': Deep_WL_Denoise_Result})
WaveFilter_AFTR_Denoising.index = ['accuracy', 'precision', 'f1', 'recall']
WaveFilter_AFTR_Denoising = WaveFilter_AFTR_Denoising.transpose()
WaveFilter_AFTR_Denoising

,accuracy,precision,f1,recall
SVM,46.4000%,45.2095%,45.6054%,46.4000%
DTree,42.4000%,40.7652%,40.3683%,42.4000%
Rforest,51.2000%,51.3668%,50.1676%,51.2000%
NB,37.6000%,33.0762%,34.0614%,37.6000%
KNN,45.6000%,46.2718%,44.4498%,45.6000%
DeepForest,50.8000%,50.1233%,48.8680%,50.8000%


# Rough Work

In [21]:
from Models.MLAlgos import Dtree

DTSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Dtree_WRDenoise_Result = Dtree(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1, split= 2)
    DTSipltResults.append(Dtree_WRDenoise_Result)
    if i>0:
        if DTSipltResults[i] == DTSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((DTSipltResults[i][0].strip('%')))/100 , float((DTSipltResults[i][1].strip('%')))/100 , float((DTSipltResults[i][2].strip('%')))/100 , float((DTSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
indexu2 = sumtol.index(max(sumtol)) + 1
print('index: ',indexu2)
print('BestResult:', DTSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
DTSipltResults

C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


index:  5
BestResult: ['42.4000%', '40.7652%', '40.3683%', '42.4000%']
[0.8821670000000001, 1.1947340000000002, 1.581381, 1.5857869999999998, 1.659335, 1.5822850000000002, 1.6471179999999999, 1.572362, 1.62801, 1.6073499999999998, 1.627444, 1.5496969999999999, 1.4469250000000002, 1.4178430000000002, 1.4197490000000001, 1.492222, 1.4310169999999998, 1.3788010000000002, 1.3649969999999998, 1.4018560000000002, 1.367375, 1.399751, 1.3986770000000002, 1.3468440000000002, 1.3819879999999998, 1.371785, 1.402319, 1.365923, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324]


[['29.6000%', '12.0747%', '16.9420%', '29.6000%'],
 ['32.0000%', '28.1045%', '27.3689%', '32.0000%'],
 ['38.8000%', '42.6378%', '37.9003%', '38.8000%'],
 ['40.0000%', '40.1824%', '38.3963%', '40.0000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['40.0000%', '39.1045%', '39.1240%', '40.0000%'],
 ['41.6000%', '40.6133%', '40.8985%', '41.6000%'],
 ['39.6000%', '38.9128%', '39.1234%', '39.6000%'],
 ['40.8000%', '40.7170%', '40.4840%', '40.8000%'],
 ['40.4000%', '39.9354%', '39.9996%', '40.4000%'],
 ['40.8000%', '40.6946%', '40.4498%', '40.8000%'],
 ['38.8000%', '38.7958%', '38.5739%', '38.8000%'],
 ['36.4000%', '35.9532%', '35.9393%', '36.4000%'],
 ['35.6000%', '35.3494%', '35.2349%', '35.6000%'],
 ['35.6000%', '35.4614%', '35.3135%', '35.6000%'],
 ['37.6000%', '36.9767%', '37.0455%', '37.6000%'],
 ['36.0000%', '35.4781%', '35.6236%', '36.0000%'],
 ['34.8000%', '33.9742%', '34.3059%', '34.8000%'],
 ['34.4000%', '33.7328%', '33.9669%', '34.4000%'],
 ['35.2000%', '34.9118%', '34.8

In [22]:
from Models.MLAlgos import SVectorMachine, Dtree

DTSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Dtree_WRDenoise_Result = Dtree(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val, deepth=indexu2, split= i+2)
    DTSipltResults.append(Dtree_WRDenoise_Result)
    if i>0:
        if DTSipltResults[i] == DTSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((DTSipltResults[i][0].strip('%')))/100 , float((DTSipltResults[i][1].strip('%')))/100 , float((DTSipltResults[i][2].strip('%')))/100 , float((DTSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
print('index: ',sumtol.index(max(sumtol)))
print('BestResult:', DTSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
DTSipltResults

index:  0
BestResult: ['42.4000%', '40.7652%', '40.3683%', '42.4000%']
[1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335]


[['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%']]

In [23]:
from Models.MLAlgos import Rforest
RFSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Rforest_WRDenoise_Result = Rforest(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1, split= 2)
    RFSipltResults.append(Rforest_WRDenoise_Result)
    if i>0:
        if RFSipltResults[i] == RFSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((RFSipltResults[i][0].strip('%')))/100 , float((RFSipltResults[i][1].strip('%')))/100 , float((RFSipltResults[i][2].strip('%')))/100 , float((RFSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
indexu = sumtol.index(max(sumtol)) + 1
print('index: ',indexu)
print('BestResult:', RFSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
RFSipltResults

index:  10
BestResult: ['51.2000%', '50.9178%', '50.0204%', '51.2000%']
[1.13232, 1.3984010000000002, 1.7051589999999999, 1.6506729999999998, 1.8506719999999999, 1.81574, 1.79608, 1.7036950000000002, 1.8765619999999998, 2.033382, 1.814002, 1.862044, 1.853728, 1.831882, 1.8916680000000001, 1.757235, 1.808194, 1.910738, 1.899544, 1.996605, 1.950023, 2.0129900000000003, 1.93683, 1.970352, 1.9979900000000002, 1.981997, 2.032146, 2.017865, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658]


[['33.2000%', '24.2860%', '22.5460%', '33.2000%'],
 ['37.2000%', '33.2911%', '32.1490%', '37.2000%'],
 ['43.6000%', '42.0776%', '41.2383%', '43.6000%'],
 ['42.0000%', '41.0833%', '39.9840%', '42.0000%'],
 ['46.8000%', '46.2338%', '45.2334%', '46.8000%'],
 ['46.4000%', '44.4753%', '44.2987%', '46.4000%'],
 ['45.2000%', '45.4089%', '43.7991%', '45.2000%'],
 ['43.2000%', '42.0419%', '41.9276%', '43.2000%'],
 ['47.2000%', '47.1862%', '46.0700%', '47.2000%'],
 ['51.2000%', '50.9178%', '50.0204%', '51.2000%'],
 ['46.0000%', '44.5525%', '44.8477%', '46.0000%'],
 ['47.2000%', '45.8734%', '45.9310%', '47.2000%'],
 ['46.4000%', '46.4924%', '46.0804%', '46.4000%'],
 ['46.8000%', '44.5819%', '45.0063%', '46.8000%'],
 ['47.6000%', '47.1008%', '46.8660%', '47.6000%'],
 ['44.4000%', '43.4724%', '43.4511%', '44.4000%'],
 ['45.6000%', '44.8970%', '44.7224%', '45.6000%'],
 ['48.4000%', '47.0293%', '47.2445%', '48.4000%'],
 ['48.0000%', '46.9885%', '46.9659%', '48.0000%'],
 ['50.4000%', '49.4688%', '49.3

In [24]:
from Models.MLAlgos import Rforest
RFSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Rforest_WRDenoise_Result = Rforest(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val, deepth=indexu, split= i+2)
    RFSipltResults.append(Rforest_WRDenoise_Result)
    if i>0:
        if RFSipltResults[i] == RFSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((RFSipltResults[i][0].strip('%')))/100 , float((RFSipltResults[i][1].strip('%')))/100 , float((RFSipltResults[i][2].strip('%')))/100 , float((RFSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
print('index: ',sumtol.index(max(sumtol)))
print('BestResult:', RFSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
RFSipltResults

index:  24
BestResult: ['51.2000%', '51.3668%', '50.1676%', '51.2000%']
[2.033382, 1.948746, 1.843778, 1.940435, 1.917601, 1.9110399999999998, 1.935161, 1.967788, 1.973826, 1.692222, 1.826684, 1.8214099999999998, 1.863004, 1.785274, 1.8358839999999998, 1.744238, 1.985206, 1.878082, 1.8720690000000002, 1.861179, 1.870384, 1.8983750000000001, 1.8529399999999998, 2.039177, 2.039344, 1.8497979999999998, 2.0046310000000003, 1.797906, 1.833192, 1.875213, 1.869775, 1.8793060000000001, 1.872906, 1.822461, 1.825398, 1.8149829999999998, 1.805987, 1.777297, 1.849055, 1.829976, 1.895251, 1.8920970000000001, 1.901371, 1.855384, 1.8380169999999998, 1.811537, 1.847176, 1.7685309999999999, 1.86825, 1.780754]


[['51.2000%', '50.9178%', '50.0204%', '51.2000%'],
 ['49.2000%', '48.3905%', '48.0841%', '49.2000%'],
 ['46.8000%', '45.4299%', '45.3479%', '46.8000%'],
 ['48.8000%', '48.5263%', '47.9172%', '48.8000%'],
 ['48.4000%', '47.5713%', '47.3888%', '48.4000%'],
 ['48.4000%', '47.1966%', '47.1074%', '48.4000%'],
 ['48.8000%', '48.1970%', '47.7191%', '48.8000%'],
 ['49.6000%', '48.9800%', '48.5988%', '49.6000%'],
 ['49.6000%', '49.6718%', '48.5108%', '49.6000%'],
 ['42.8000%', '42.0938%', '41.5284%', '42.8000%'],
 ['46.0000%', '45.4276%', '45.2408%', '46.0000%'],
 ['46.0000%', '45.3155%', '44.8255%', '46.0000%'],
 ['47.2000%', '46.0446%', '45.8558%', '47.2000%'],
 ['45.2000%', '44.2369%', '43.8905%', '45.2000%'],
 ['46.4000%', '45.7621%', '45.0263%', '46.4000%'],
 ['44.0000%', '43.3034%', '43.1204%', '44.0000%'],
 ['50.0000%', '49.4069%', '49.1137%', '50.0000%'],
 ['47.2000%', '46.9458%', '46.4624%', '47.2000%'],
 ['47.2000%', '46.6910%', '46.1159%', '47.2000%'],
 ['46.8000%', '46.5625%', '45.9

In [32]:
Dtree_WL_Denoise_Result = Dtree(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,deepth =5 ,split=2)
Rforest_WL_Denoise_Result = Rforest(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,deepth =10 ,split=26)

In [33]:
Dtree_WL_Denoise_Result, Rforest_WL_Denoise_Result

(['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['51.2000%', '51.3668%', '50.1676%', '51.2000%'])

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = { 'C':[0.1,1,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6]}

In [26]:
grid = GridSearchCV(SVC(random_state=10),param_grid)
grid.fit(X_Wavlet_Denoise_Embed_train, Y_train)

GridSearchCV(estimator=SVC(random_state=10),
             param_grid={'C': [0.1, 1, 100, 1000], 'degree': [1, 2, 3, 4, 5, 6],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [27]:
print(grid.best_params_)
print(grid.score(X_Wavlet_Denoise_Embed_val, Y_val))

{'C': 1000, 'degree': 1, 'kernel': 'rbf'}
0.464


In [35]:
SVM_WL_Denoise_Result = SVectorMachine(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,Cval=1000, deguree=1, kernelu ='rbf')
SVM_WL_Denoise_Result

['46.4000%', '45.2095%', '45.6054%', '46.4000%']

In [30]:
SVM_WL_Denoise_Result = SVectorMachine(X_Wavlet_Denoise_Embed_train, Y_train, X_Wavlet_Denoise_Embed_val, Y_val,Cval=grid.best_params_['C'], deguree=grid.best_params_['degree'], kernelu =grid.best_params_['kernel'])

In [31]:
SVM_WL_Denoise_Result

['46.4000%', '45.2095%', '45.6054%', '46.4000%']

In [37]:
from Models.MLAlgos import Dtree

DTSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Dtree_WRDenoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1, split= 2)
    DTSipltResults.append(Dtree_WRDenoise_Result)
    if i>0:
        if DTSipltResults[i] == DTSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((DTSipltResults[i][0].strip('%')))/100 , float((DTSipltResults[i][1].strip('%')))/100 , float((DTSipltResults[i][2].strip('%')))/100 , float((DTSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
indexu2 = sumtol.index(max(sumtol)) + 1
print('index: ',sumtol.index(max(sumtol)))
print('BestResult:', DTSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
DTSipltResults

C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


index:  4
BestResult: ['42.4000%', '40.7652%', '40.3683%', '42.4000%']
[0.8821670000000001, 1.1947340000000002, 1.581381, 1.5857869999999998, 1.659335, 1.5822850000000002, 1.6471179999999999, 1.572362, 1.62801, 1.6073499999999998, 1.627444, 1.5496969999999999, 1.4469250000000002, 1.4178430000000002, 1.4197490000000001, 1.492222, 1.4310169999999998, 1.3788010000000002, 1.3649969999999998, 1.4018560000000002, 1.367375, 1.399751, 1.3986770000000002, 1.3468440000000002, 1.3819879999999998, 1.371785, 1.402319, 1.365923, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324, 1.32324]


[['29.6000%', '12.0747%', '16.9420%', '29.6000%'],
 ['32.0000%', '28.1045%', '27.3689%', '32.0000%'],
 ['38.8000%', '42.6378%', '37.9003%', '38.8000%'],
 ['40.0000%', '40.1824%', '38.3963%', '40.0000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['40.0000%', '39.1045%', '39.1240%', '40.0000%'],
 ['41.6000%', '40.6133%', '40.8985%', '41.6000%'],
 ['39.6000%', '38.9128%', '39.1234%', '39.6000%'],
 ['40.8000%', '40.7170%', '40.4840%', '40.8000%'],
 ['40.4000%', '39.9354%', '39.9996%', '40.4000%'],
 ['40.8000%', '40.6946%', '40.4498%', '40.8000%'],
 ['38.8000%', '38.7958%', '38.5739%', '38.8000%'],
 ['36.4000%', '35.9532%', '35.9393%', '36.4000%'],
 ['35.6000%', '35.3494%', '35.2349%', '35.6000%'],
 ['35.6000%', '35.4614%', '35.3135%', '35.6000%'],
 ['37.6000%', '36.9767%', '37.0455%', '37.6000%'],
 ['36.0000%', '35.4781%', '35.6236%', '36.0000%'],
 ['34.8000%', '33.9742%', '34.3059%', '34.8000%'],
 ['34.4000%', '33.7328%', '33.9669%', '34.4000%'],
 ['35.2000%', '34.9118%', '34.8

In [39]:
from Models.MLAlgos import SVectorMachine, Dtree

DTSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Dtree_WRDenoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=indexu2, split= i+2)
    DTSipltResults.append(Dtree_WRDenoise_Result)
    if i>0:
        if DTSipltResults[i] == DTSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((DTSipltResults[i][0].strip('%')))/100 , float((DTSipltResults[i][1].strip('%')))/100 , float((DTSipltResults[i][2].strip('%')))/100 , float((DTSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
print('index: ',sumtol.index(max(sumtol)))
print('BestResult:', DTSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
DTSipltResults

index:  0
BestResult: ['42.4000%', '40.7652%', '40.3683%', '42.4000%']
[1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335, 1.659335]


[['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%'],
 ['42.4000%', '40.7652%', '40.3683%', '42.4000%']]

In [41]:
Dtree_MuRIL_WL_Denoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=5, split=2)
Dtree_MuRIL_WL_Denoise_Result

['42.4000%', '40.7652%', '40.3683%', '42.4000%']

In [30]:
from Models.MLAlgos import Rforest
RFSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Rforest_WRDenoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1, split= 2)
    RFSipltResults.append(Rforest_WRDenoise_Result)
    if i>0:
        if RFSipltResults[i] == RFSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((RFSipltResults[i][0].strip('%')))/100 , float((RFSipltResults[i][1].strip('%')))/100 , float((RFSipltResults[i][2].strip('%')))/100 , float((RFSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
indexu = sumtol.index(max(sumtol)) + 1
print('index: ',indexu)
print('BestResult:', RFSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
RFSipltResults

index:  10
BestResult: ['51.2000%', '50.9178%', '50.0204%', '51.2000%']
[1.13232, 1.3984010000000002, 1.7051589999999999, 1.6506729999999998, 1.8506719999999999, 1.81574, 1.79608, 1.7036950000000002, 1.8765619999999998, 2.033382, 1.814002, 1.862044, 1.853728, 1.831882, 1.8916680000000001, 1.757235, 1.808194, 1.910738, 1.899544, 1.996605, 1.950023, 2.0129900000000003, 1.93683, 1.970352, 1.9979900000000002, 1.981997, 2.032146, 2.017865, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658, 2.030658]


[['33.2000%', '24.2860%', '22.5460%', '33.2000%'],
 ['37.2000%', '33.2911%', '32.1490%', '37.2000%'],
 ['43.6000%', '42.0776%', '41.2383%', '43.6000%'],
 ['42.0000%', '41.0833%', '39.9840%', '42.0000%'],
 ['46.8000%', '46.2338%', '45.2334%', '46.8000%'],
 ['46.4000%', '44.4753%', '44.2987%', '46.4000%'],
 ['45.2000%', '45.4089%', '43.7991%', '45.2000%'],
 ['43.2000%', '42.0419%', '41.9276%', '43.2000%'],
 ['47.2000%', '47.1862%', '46.0700%', '47.2000%'],
 ['51.2000%', '50.9178%', '50.0204%', '51.2000%'],
 ['46.0000%', '44.5525%', '44.8477%', '46.0000%'],
 ['47.2000%', '45.8734%', '45.9310%', '47.2000%'],
 ['46.4000%', '46.4924%', '46.0804%', '46.4000%'],
 ['46.8000%', '44.5819%', '45.0063%', '46.8000%'],
 ['47.6000%', '47.1008%', '46.8660%', '47.6000%'],
 ['44.4000%', '43.4724%', '43.4511%', '44.4000%'],
 ['45.6000%', '44.8970%', '44.7224%', '45.6000%'],
 ['48.4000%', '47.0293%', '47.2445%', '48.4000%'],
 ['48.0000%', '46.9885%', '46.9659%', '48.0000%'],
 ['50.4000%', '49.4688%', '49.3

In [31]:
from Models.MLAlgos import Rforest
RFSipltResults = []
sumtol = []
count = 0
for i in range(50):
    Rforest_WRDenoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=indexu, split= i+2)
    RFSipltResults.append(Rforest_WRDenoise_Result)
    if i>0:
        if RFSipltResults[i] == RFSipltResults[i-1]:
            count = count + 1

    sumtemp = sum([float((RFSipltResults[i][0].strip('%')))/100 , float((RFSipltResults[i][1].strip('%')))/100 , float((RFSipltResults[i][2].strip('%')))/100 , float((RFSipltResults[i][3].strip('%')))/100])
    sumtol.append(sumtemp)
    if count == 10:
        break
print('index: ',sumtol.index(max(sumtol)))
print('BestResult:', RFSipltResults[sumtol.index(max(sumtol))])
print(sumtol)
RFSipltResults

index:  24
BestResult: ['51.2000%', '51.3668%', '50.1676%', '51.2000%']
[2.033382, 1.948746, 1.843778, 1.940435, 1.917601, 1.9110399999999998, 1.935161, 1.967788, 1.973826, 1.692222, 1.826684, 1.8214099999999998, 1.863004, 1.785274, 1.8358839999999998, 1.744238, 1.985206, 1.878082, 1.8720690000000002, 1.861179, 1.870384, 1.8983750000000001, 1.8529399999999998, 2.039177, 2.039344, 1.8497979999999998, 2.0046310000000003, 1.797906, 1.833192, 1.875213, 1.869775, 1.8793060000000001, 1.872906, 1.822461, 1.825398, 1.8149829999999998, 1.805987, 1.777297, 1.849055, 1.829976, 1.895251, 1.8920970000000001, 1.901371, 1.855384, 1.8380169999999998, 1.811537, 1.847176, 1.7685309999999999, 1.86825, 1.780754]


[['51.2000%', '50.9178%', '50.0204%', '51.2000%'],
 ['49.2000%', '48.3905%', '48.0841%', '49.2000%'],
 ['46.8000%', '45.4299%', '45.3479%', '46.8000%'],
 ['48.8000%', '48.5263%', '47.9172%', '48.8000%'],
 ['48.4000%', '47.5713%', '47.3888%', '48.4000%'],
 ['48.4000%', '47.1966%', '47.1074%', '48.4000%'],
 ['48.8000%', '48.1970%', '47.7191%', '48.8000%'],
 ['49.6000%', '48.9800%', '48.5988%', '49.6000%'],
 ['49.6000%', '49.6718%', '48.5108%', '49.6000%'],
 ['42.8000%', '42.0938%', '41.5284%', '42.8000%'],
 ['46.0000%', '45.4276%', '45.2408%', '46.0000%'],
 ['46.0000%', '45.3155%', '44.8255%', '46.0000%'],
 ['47.2000%', '46.0446%', '45.8558%', '47.2000%'],
 ['45.2000%', '44.2369%', '43.8905%', '45.2000%'],
 ['46.4000%', '45.7621%', '45.0263%', '46.4000%'],
 ['44.0000%', '43.3034%', '43.1204%', '44.0000%'],
 ['50.0000%', '49.4069%', '49.1137%', '50.0000%'],
 ['47.2000%', '46.9458%', '46.4624%', '47.2000%'],
 ['47.2000%', '46.6910%', '46.1159%', '47.2000%'],
 ['46.8000%', '46.5625%', '45.9

In [32]:
Rforest_WRDenoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=10, split= 26)
Rforest_WRDenoise_Result

['51.2000%', '51.3668%', '50.1676%', '51.2000%']

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = { 'C':[0.1,1,10,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6]}

In [11]:
grid = GridSearchCV(SVC(random_state=10),param_grid)
grid.fit(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train)

GridSearchCV(estimator=SVC(random_state=10),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [13]:
print(grid.best_params_)
print(grid.score(X_MuRIL_Wavlet_Denoise_Embed_val, Y_val))

{'C': 1000, 'degree': 1, 'kernel': 'rbf'}
0.464


In [28]:
SVM_MuRIL_WL_Denoise_Result = SVectorMachine(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, Cval = 1000, deguree = 1)
SVM_MuRIL_WL_Denoise_Result

['46.4000%', '45.2095%', '45.6054%', '46.4000%']

In [9]:
import random
random.seed(10)

from Models.MLAlgos import Dtree, Rforest

DT_Results = []
RF_Results = []

for i in range(50):
    Dtree_MuRIL_WL_Denoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1)
    DT_Results.append(Dtree_MuRIL_WL_Denoise_Result)
    Rforest_MuRIL_WL_Denoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=i+1)
    RF_Results.append(Rforest_MuRIL_WL_Denoise_Result)

C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
DT_Results

[['31.8000%', '13.9674%', '18.9383%', '31.8000%'],
 ['32.4000%', '19.6857%', '23.0952%', '32.4000%'],
 ['35.4000%', '30.9388%', '26.7417%', '35.4000%'],
 ['37.4000%', '36.0342%', '34.7707%', '37.4000%'],
 ['34.8000%', '35.7545%', '34.2306%', '34.8000%'],
 ['33.8000%', '33.5469%', '32.9029%', '33.8000%'],
 ['34.0000%', '37.5902%', '34.6368%', '34.0000%'],
 ['32.8000%', '34.7466%', '33.2292%', '32.8000%'],
 ['36.8000%', '37.7867%', '37.1605%', '36.8000%'],
 ['35.6000%', '36.3158%', '35.7602%', '35.6000%'],
 ['33.8000%', '34.6525%', '34.0054%', '33.8000%'],
 ['33.6000%', '33.8872%', '33.6499%', '33.6000%'],
 ['33.6000%', '34.3251%', '33.7534%', '33.6000%'],
 ['34.6000%', '34.7104%', '34.5550%', '34.6000%'],
 ['34.4000%', '34.5759%', '34.4595%', '34.4000%'],
 ['32.8000%', '32.8596%', '32.8033%', '32.8000%'],
 ['34.0000%', '34.4985%', '34.1858%', '34.0000%'],
 ['32.2000%', '32.3071%', '32.2096%', '32.2000%'],
 ['33.0000%', '33.5627%', '33.2331%', '33.0000%'],
 ['31.6000%', '32.1457%', '31.8

In [11]:
RF_Results

[['36.8000%', '30.8401%', '28.8544%', '36.8000%'],
 ['38.6000%', '34.8776%', '31.9249%', '38.6000%'],
 ['41.4000%', '40.6474%', '36.7837%', '41.4000%'],
 ['42.0000%', '40.7901%', '39.0335%', '42.0000%'],
 ['44.0000%', '41.2179%', '41.0346%', '44.0000%'],
 ['45.0000%', '42.5812%', '43.0802%', '45.0000%'],
 ['47.4000%', '45.3046%', '45.2885%', '47.4000%'],
 ['49.0000%', '48.0650%', '47.4266%', '49.0000%'],
 ['48.6000%', '47.1690%', '47.2431%', '48.6000%'],
 ['46.8000%', '45.0440%', '45.5459%', '46.8000%'],
 ['46.6000%', '44.5130%', '45.1482%', '46.6000%'],
 ['45.2000%', '43.5205%', '44.1146%', '45.2000%'],
 ['48.2000%', '46.8562%', '46.9857%', '48.2000%'],
 ['45.4000%', '43.3812%', '44.0522%', '45.4000%'],
 ['47.6000%', '46.2118%', '46.5490%', '47.6000%'],
 ['48.6000%', '46.5654%', '47.1710%', '48.6000%'],
 ['45.8000%', '44.3541%', '44.8308%', '45.8000%'],
 ['46.0000%', '44.2141%', '44.6897%', '46.0000%'],
 ['45.4000%', '43.5645%', '44.1245%', '45.4000%'],
 ['45.4000%', '43.5667%', '44.1

In [9]:
import random
random.seed(10)
# When sigma= 0.1 then deepth is 9(DT),8(RF)

from Models.MLAlgos import Dtree, Rforest

Dtree_MuRIL_WL_Denoise_Result = Dtree(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=9)
Rforest_MuRIL_WL_Denoise_Result = Rforest(X_MuRIL_Wavlet_Denoise_Embed_train, Y_train, X_MuRIL_Wavlet_Denoise_Embed_val, Y_val, deepth=8)

Dtree_MuRIL_WL_Denoise_Result,Rforest_MuRIL_WL_Denoise_Result

(['36.8000%', '37.7867%', '37.1605%', '36.8000%'],
 ['49.0000%', '48.0650%', '47.4266%', '49.0000%'])